In [ ]:
# Importing relevant python packages
import pandas as pd
import gdal
import numpy as np
import time, os

# For plotting
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib.patches as patches

font = {'family' : 'monospace',
        'weight' : 'bold',
        'size'   : 18}
plt.rc('font', **font)

In [ ]:
datadirectory = '/Volumes/Salo/Salo_ai/SAR_CH3/wa_v2/cra'
datefile = 'S32631X402380Y1491460sS1_A_vv_0001_A_mtfil.dates'
imagefile='S32631X402380Y1491460sS1_A_vv_0001_A_mtfil.vrt'

In [ ]:
# Switch to the data directory
os.chdir(datadirectory)

In [ ]:
dates = open(datefile).readlines()
tindex = pd.DatetimeIndex(dates)
j = 1
print('Bands and dates for', imagefile)
for i in tindex:
    print("{:4d} {}".format(j, i.date()), end = ' ')
    j+=1
    if j % 5 == 1: print()

In [ ]:
rasterstack = gdal.Open(imagefile).ReadAsArray()

In [ ]:
# 1. Converstion to Power
caldB = -83
calPwr = np.power(10., caldB/10.)
rasterstack_pwr = np.power(rasterstack, 2.) * calPwr
# 2. compute Means
rs_means_pwr = np.mean(rasterstack_pwr, axis = (1, 2))
# 3. Convert to dB
rs_means_dB = 10.*np.log10(rs_means_pwr)

In [ ]:
# 4. Make a pandas time series object
ts = pd.Series(rs_means_dB, index = tindex)

In [ ]:
# 5. Use the pandas plot function of the time series object to plot 
# Put band numbers as data point labels
plt.figure(figsize = (16, 8))
ts.plot()
x1 = plt.xlabel('Date')
y1 = plt.ylabel('$\overline{\gamma^o}$ [dB]')
for xyb in zip(ts.index, rs_means_dB, range(1, len(ts)+1)):
    plt.annotate(xyb[2], xy = xyb[0:2])

In [ ]:
def showImage(rasterstack,tindex,bandnbr,subset=None,vmin=None,vmax=None):
    '''Input:
    rasterstack stack of images in SAR power units
    tindex time series date index
    bandnbr bandnumber of the rasterstack to dissplay'''
    fig = plt.figure(figsize=(16,8))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    # If vmin or vmax are None we use percentiles as limits:
    if vmin==None: vmin=np.percentile(rasterstack[bandnbr-1].flatten(),5)
    if vmax==None: vmax=np.percentile(rasterstack[bandnbr-1].flatten(),95)
    ax1.imshow(rasterstack[bandnbr-1],cmap='gray',vmin=vmin,vmax=vmax)
    ax1.set_title('Image Band {} {}'.format(bandnbr,tindex[bandnbr-1].date()))
    if subset== None:
        bands,ydim,xdim=rasterstack.shape
        subset=(0,0,xdim,ydim)
    
    ax1.add_patch(patches.Rectangle((subset[0],subset[1]),subset[2],subset[3],fill=False,edgecolor='red'))
    ax1.xaxis.set_label_text('Pixel')
    ax1.yaxis.set_label_text('Line')
    ts_pwr=np.mean(rasterstack[:,subset[1]:(subset[1]+subset[3]),
                               subset[0]:(subset[0]+subset[2])],axis=(1,2))
    ts_dB=10.*np.log10(ts_pwr)
    ax2.plot(tindex,ts_dB)
    ax2.yaxis.set_label_text('$\gamma^o$ [dB]')
    ax2.set_title('$\gamma^o$ Backscatter Time Series')
    # Add a vertical line for the date where the image is displayed
    ax2.axvline(tindex[bandnbr-1],color='red')
    fig.autofmt_xdate()

In [ ]:
def timeSeries(rasterstack_pwr,tindex,subset,ndv=0.):
    # Extract the means along the time series axes
    # raster shape is time steps, lines, pixels.
    # With axis=1,2, we average lines and pixels for each time
    # step (axis 0)
    raster=rasterstack_pwr.copy()
    if ndv != np.nan: raster[np.equal(raster,ndv)]=np.nan
    ts_pwr=np.nanmean(raster[:,subset[1]:(subset[1]+subset[3]),
                             subset[0]:(subset[0]+subset[2])],axis=(1,2))
    # convert the means to dB
    ts_dB=10.*np.log10(ts_pwr)
    # make the pandas time series object
    ts = pd.Series(ts_dB,index=tindex)
    # return it
    return ts

In [ ]:
subset = (5, 20, 5, 5)
ts = timeSeries(rasterstack_pwr, tindex, subset)
tsdf = pd.DataFrame(ts, index = ts.index, columns = ['g0'])

#  Plot 
ylim = (-20, -5)
tsdf.plot(figsize = (16,4))
plt.title('Sentinel-1 C-VV Time Series Backscatter Profile, Subset: 5, 20, 5, 5  ')
plt.ylabel('$\gamma^o$ [dB]')
plt.ylim(ylim)
_ = plt.legend(["C-VV Time Series"])

In [ ]:
tsdf_sub1 = tsdf[tsdf.index>'2015-11-01']

#  Plot
tsdf_sub1.plot(figsize  = (16,4))
plt.title('Sentinel-1 C-VV Time Series Backscatter Profile, Subset: {}'.format(subset))
plt.ylabel('$\gamma^o$ [dB]')
plt.ylim(ylim)
_ = plt.legend(["C-VV Time Series"])

In [ ]:
tsdf_sub2 = tsdf_sub1[
    np.logical_and(tsdf_sub1.index.month>=3, tsdf_sub1.index.month<=5)
]

# Plot 
fig, ax = plt.subplots(figsize = (16,4))
tsdf_sub2.plot(ax = ax)
plt.title('Sentinel-1 C-VV Time Series Backscatter Profile, Subset: {}'.format(subset))
plt.ylabel('$\gamma^o$ [dB]')
plt.ylim(ylim)
_ = plt.legend(['March-May'])

In [ ]:
tsdf_sub3 = tsdf_sub1[np.invert(
    np.logical_and(tsdf_sub1.index.month>=3, tsdf_sub1.index.month<=5))]

# Plot
fig, ax = plt.subplots(figsize = (16,4))
tsdf_sub3.plot(ax = ax)
plt.title('Sentinel-1 C-VV Time Series Backscatter Profile, Subset: {}'.format(subset))
plt.ylim(ylim)
plt.ylabel('$\gamma^o$ [dB]')
_ = plt.legend(["June - February"])

In [ ]:
ts_sub_by_year = tsdf_sub1.groupby(pd.Grouper(freq = "Y"))

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
for label, df in ts_sub_by_year:
    df.g0.plot(ax = ax, label = label.year)
plt.legend()
# ts_sub_by_year.plot(ax = ax)
plt.title('Sentinel-1 C-VV Time Series Backscatter Profile, Subset: {}'.format(subset))
plt.ylabel('$\gamma^o$ [dB]')
plt.ylim(ylim)

In [ ]:
# Add doy
tsdf_sub1 = tsdf_sub1.assign(doy = tsdf_sub1.index.dayofyear)
# Add Year
tsdf_sub1 = tsdf_sub1.assign(year = tsdf_sub1.index.year)

In [ ]:
piv = pd.pivot_table(tsdf_sub1, index = ['doy'], columns = ['year'], values = ['g0'])
# Set the names for the column indices
piv.columns.set_names(['g0', 'Year'],inplace = True)
print(piv.head(10))
print('...\n', piv.tail(10))

In [ ]:
piv.columns.set_names(['g0', 'year'], inplace = True)

In [ ]:
# Add fake dates for year 1-- to enable time sensitive interpolation 
# of missing values in the pivot table
year_doy = ['2100-{}'.format(x) for x in piv.index]
y100_doy = pd.DatetimeIndex(pd.to_datetime(year_doy, format='%Y-%j'))

# make a copy of the piv table and add two columns
piv2 = piv.copy()
piv2 = piv2.assign(d100 = y100_doy) # add the fake year dates
piv2 = piv2.assign(doy = piv2.index) # add doy as a colum to replace as index later again

# Set the index to the dummy year
piv2.set_index('d100', inplace = True, drop = True)

# PERFORM THE TIME WEIGHTED INTERPOLATION
piv2 = piv2.interpolate(method = 'time') #TIME WEIGHTED INTERPOLATION!

# Set the index back to day of year
piv2.set_index('doy', inplace = True, drop = True)

In [ ]:
print(piv2.head(10))
print('...\n',piv2.tail(10))

In [ ]:
piv2.plot(figsize = (16,8))
plt.title('Sentinel-1 C-VV Time Series backscatter Profile,\
Subset: 5, 20, 5, 5  ')
plt.ylabel('$\gamma^o$ [dB]')
plt.xlabel('Day of Year')
_ = plt.ylim(ylim)

In [ ]:
# Difference between years
# Set a dB change threshold
thres = 3

In [ ]:
diff1716 = (piv2.g0[2017]-piv2.g0[2016])

In [ ]:
_ = diff1716.plot(kind = 'line')

In [ ]:
thres_exceeded = diff1716[abs(diff1716) > thres]
thres_exceeded

In [ ]:
subset = (5, 20 , 3, 3)

# subset = (12, 5, 3, 3)
ts1 = timeSeries(rasterstack_pwr, tindex, subset)
X = ts1[ts1.index>'2015-10-31']

In [ ]:
Xr = X.rolling(5, center = True).median()
Xr.plot()
_= X.plot()

In [ ]:
X = Xr # Uncomment if rolling mean is wanted for further computation
Xmean = X.mean()

In [ ]:
fig, ax = plt.subplots(figsize = (16,4))
X.plot()
plt.ylabel('$\gamma^o$ [dB]')
ax.axhline(Xmean, color = 'red')
_ = plt.legend(['$\gamma^o$','$\overline{\gamma^o}$'])

In [ ]:
R = X - Xmean

In [ ]:
S = R.cumsum()

_ = S.plot(figsize = (16,8))

In [ ]:
Sdiff = S.max() - S.min()
Sdiff

In [ ]:
t_cp_before = S[S == S.max()].index[0]
print('Last date before change: {}'.format(t_cp_before.date()))

In [ ]:
t_cp_after = S[S.index > t_cp_before].index[0]
print('First date after change: {}'.format(t_cp_after.date()))

In [ ]:
n_bootstraps = 500 # bootsrap sample size
fig, ax =plt.subplots(figsize = (16,8))
S.plot(ax = ax, linewidth = 3)
ax.set_ylabel('Cumulative Sums of the Residuals')
fig.legend(['S Curve for Candidate Change Point'],loc = 3)
Sdiff_random_sum = 0
Sdiff_random_max = 0 # To keep track of the maxium Sdiff of the bootstrapped sample
n_Sdiff_gt_Sdiff_random = 0 # To keep track of the maxiu Sdiff of the bootstrapped sample

for i in range(n_bootstraps):
    Rrandom = R.sample(frac = 1) # Randomize the time steps of the residuals
    Srandom = Rrandom.cumsum()
    Sdiff_random = Srandom.max()-Srandom.min()
    Sdiff_random_sum += Sdiff_random
    if Sdiff_random > Sdiff_random_max:
        Sdiff_random_max = Sdiff_random
    if Sdiff > Sdiff_random:
        n_Sdiff_gt_Sdiff_random +=1
    Srandom.plot(ax = ax)
_ = ax.axhline(Sdiff_random_sum/n_bootstraps)

In [ ]:
CL = 1.*n_Sdiff_gt_Sdiff_random/n_bootstraps
print('Confidence Level for change point {} percent'.format(CL*100.))

In [ ]:
CP_significance = 1. - (Sdiff_random_sum/n_bootstraps)/Sdiff
print('Change point significance metric: {}'.format(CP_significance))

In [ ]:
# NaN's to be excluded in the computation
S_ni = (S.abs()/S.abs().max()).cumsum().max()/len(S[S != np.nan])
print('Normalized Integral of cumulative sum: {}'.format(S_ni))

In [ ]:
means_pwr = np.mean(rasterstack_pwr, axis = (1, 2))
means_dB = 10.*np.log10(means_pwr)
gm_ts = pd.Series(means_dB, index = tindex)
gm_ts = gm_ts[gm_ts.index > '2015-10-31'] # filter dates
gm_ts = gm_ts.rolling(5, center = True).median()

In [ ]:
gm_ts.plot()

In [ ]:
X.plot()

In [ ]:
Xd = X - gm_ts
Xmean = Xd.mean()
Xd.plot()

In [ ]:
S = R.cumsum()

_ = S.plot(figsize = (16,8))

In [ ]:
Sdiff = S.max() - S.min()
Sdiff

In [ ]:
X = rasterstack_pwr
# Filter out the first layer (Dates >= '2015-11-1')
X_sub = X[1:,:,:]
tindex_sub = tindex[1:]
X = 10.*np.log10(X_sub)

In [ ]:
plt.figure()
# Indicate the band number
bandnbr = 0
vmin = np.percentile(X[bandnbr],5)
vmax = np.percentile(X[bandnbr],95)
plt.title('Band {} {}'.format(bandnbr+1, tindex_sub[bandnbr].date()))
plt.imshow(X[0], cmap = 'gray', vmin = vmin, vmax = vmax)
_ = plt.colorbar()

In [ ]:
Xmean = np.mean(X, axis = 0)
plt.figure()
plt.imshow(Xmean, cmap = 'gray')

In [ ]:
X.shape

In [ ]:
R = X - Xmean

In [ ]:
plt.imshow(R[0])
plt.title('Residuals')
_ = plt.colorbar()

In [ ]:
S = np.cumsum(R,axis = 0)
Smax = np.max(S, axis = 0)
Smin = np.min(S, axis = 0)
Sdiff = Smax - Smin
fig, ax = plt.subplots(1,3,figsize = (16,4))
vmin = Smin.min()
vmax = Smax.max()
p = ax[0].imshow(Smin, vmin = vmin, vmax = vmax)
ax[0].set_title('$S_{max}$')
ax[1].imshow(Smin, vmin = vmin, vmax = vmax)
ax[1].set_title('$S_{min}$')
ax[2].imshow(Sdiff, vmin = vmin, vmax = vmax)
ax[2].set_title('$S_{diff}$')
fig.subplots_adjust(right = 0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
_ = fig.colorbar(p, cax = cbar_ax)

In [ ]:
# Display the Sdiff histogram
precentile = 50
fig, ax = plt.subplots()
h = ax.hist(Sdiff.flatten(), bins = 50)
thres = np.percentile(h[1],50)
print('At the {}% percentile, the threshold value is {:2.2f}'.format(precentile, thres))
_ = ax.axvline(thres, color = 'red')

In [ ]:
Sdiffmask = Sdiff<thres
_ = plt.imshow(Sdiffmask, cmap = 'gray')

In [ ]:
Rmask = np.broadcast_to(Sdiffmask, R.shape)

In [ ]:
Rmasked = np.ma.array(R, mask = Rmask)

In [ ]:
Smasked = np.ma.cumsum(Rmasked, axis = 0)

In [ ]:
plt.imshow(Rmasked.mask[0], cmap = 'gray')

In [ ]:
Smasked = np.ma.cumsum(Rmasked, axis = 0)
Smasked_max = np.ma.max(Smasked, axis = 0)
Smasked_min = np.ma.min(Smasked, axis = 0)
Smasked_diff = Smasked_max - Smasked_min
fig, ax = plt.subplots(1,3,figsize = (16,4))
vmin = Smasked_min.min()
vmax = Smasked_max.max()
p = ax[0].imshow(Smasked_max, vmin = vmin, vmax = vmax)
ax[0].set_title('$S_{max}$')
ax[1].imshow(Smasked_min, vmin = vmin, vmax = vmax)
ax[1].set_title('$S_{min}$')
ax[2].imshow(Smasked_diff, vmin = vmin, vmax = vmax)
ax[2].set_title('$S_{diff}$')
fig.subplots_adjust(right = 0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
_ = fig.colorbar(p, cax = cbar_ax)

In [ ]:
random_index = np.random.permutation(Rmasked.shape[0])
Rrandom= Rmasked[random_index, :, :]

fig, ax = plt.subplots(1,2,figsize = (8, 4))
ax[0].imshow(Rmasked[0])
ax[0].set_title('Band 0')
ax[1].imshow(Rrandom[0])
_ = ax[1].set_title('Band 0 Randomized')

In [ ]:
Smasked_max = np.ma.max(Smasked, axis = 0)

In [ ]:
n_bootstraps=1000 # bootstrap sample size
# to keep track of the maxium Sdiff of the bootstrapped sample:
Sdiff_random_max = np.ma.copy(Smasked_diff)
Sdiff_random_max[~Sdiff_random_max.mask]=0
# to compute the Sdiff sums of the bootstrapped sample:
Sdiff_random_sum = np.ma.copy(Smasked_diff)
Sdiff_random_sum[~Sdiff_random_max.mask]=0
# to keep track of the count of the bootstrapped sample
n_Sdiff_gt_Sdiff_random = np.ma.copy(Smasked_diff)
n_Sdiff_gt_Sdiff_random[~n_Sdiff_gt_Sdiff_random.mask]=0
for i in range(n_bootstraps):
    # For efficiency, we shuffle the time axis index and use that
    #to randomize the masked array
    random_index=np.random.permutation(Rmasked.shape[0])
    # Randomize the time step of the residuals
    Rrandom = Rmasked[random_index,:,:]
    Srandom = np.ma.cumsum(Rrandom,axis=0)
    Srandom_max=np.ma.max(Srandom,axis=0)
    Srandom_min=np.ma.min(Srandom,axis=0)
    Sdiff_random=Srandom_max-Srandom_min
    Sdiff_random_sum += Sdiff_random
    Sdiff_random_max[np.ma.greater(Sdiff_random,Sdiff_random_max)]=\
    Sdiff_random[np.ma.greater(Sdiff_random,Sdiff_random_max)]
    n_Sdiff_gt_Sdiff_random[np.ma.greater(Smasked_diff,Sdiff_random)] += 1

In [ ]:
CL = n_Sdiff_gt_Sdiff_random/n_bootstraps
CP_significance = 1.-(Sdiff_random_sum/n_bootstraps)/Sdiff
# Plot
fig, ax = plt.subplots(1,3, figsize = (16,4))
a = ax[0].imshow(CL*100)
fig.colorbar(a, ax = ax[0])
ax[0].set_title('Confidence Level %')
a = ax[1].imshow(CP_significance)
fig.colorbar(a, ax = ax[1])
ax[1].set_title('Significance')
a = ax[2].imshow(CL*CP_significance)
fig.colorbar(a, ax = ax[2])
_ = ax[2].set_title('CL x S')

In [ ]:
cp_thres = 0.5

In [ ]:
plt.imshow(CL*CP_significance < cp_thres, cmap = 'cool')

In [ ]:
cp_mask = np.ma.mask_or(CL*CP_significance < cp_thres, CL.mask)
cp_mask2 = np.broadcast_to(cp_mask, Smasked.shape)
CPraster = np.ma.array(Smasked.data, mask = cp_mask2)

In [ ]:
CP_index = np.ma.argmax(CPraster, axis = 0)
change_indices = list(np.unique(CP_index))
change_indices.remove(0)
print(change_indices)
# look up the dates from the indices to get the change dates
alldates = tindex[tindex>'2015-10-31']
change_dates = [str(alldates[x+1].date()) for x in change_indices]
print(change_dates)

In [ ]:
ticks = change_indices
ticklabels = change_dates

cmap = plt.cm.get_cmap('magma', ticks[-1])
fig, ax = plt.subplots(figsize = (8,8))
cax = ax.imshow(CP_index, interpolation = 'nearest', cmap = cmap)

ax.set_title('Dates of Change')

cbar = fig.colorbar(cax, ticks = ticks, orientation = 'horizontal')
_ = cbar.ax.set_xticklabels(ticklabels, size = 10, rotation = 45, ha = 'right')